In [1]:
from app.Transformer_Classifier import Transformer_Classifier 
from app.Data_Loader import Data_Loader
from app.TextRank_Extractor import TextRank_Extractor
from app.Keyword_Classifier import Keyword_Classifier
from app.common.MySQLUtility import MySQLUtility
import os 
from app.Risk_Score_Service import Risk_Score_Service

domains = ['esg', 'liabilities' ] #'liabilities', 'esg'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './config/gcp/genuine-wording-key.json'

DB_HOST = '34.170.168.203'
DB_USER = 'root'
DB_PASSWORD = 'nu123456'
DB_NAME = 'lca_dev'

class Data_ETL_Pipeline(object):
    dbutil = None
    data_load = None
    textrank = None 
    key_classifier = None
    class_service = None
    risk_class = None

    def __init__(self):
        self.dbutil = MySQLUtility(DB_HOST, DB_USER, DB_PASSWORD, DB_NAME)
        self.data_load = Data_Loader(self.dbutil)
        self.textrank = TextRank_Extractor(self.dbutil)
        self.key_classifier = Keyword_Classifier(self.dbutil)
        self.class_service = Transformer_Classifier(self.dbutil, domains)
        self.risk_class = Risk_Score_Service(self.dbutil, domains)
        pass    

    def create_dataset(self):
        print("dbutil.db_cleanup():")
        self.dbutil.clean_db()
        print("dbutil.create_database():")
        self.dbutil.create_database() 

    def load_file_data(self): 
        print("data_load.import_seed_data_batch():")
        self.data_load.import_seed_data_batch()

        for domain in self.domains:
            print("self.data_load.import_reports_contract_data()" + domain)
            self.data_load.import_reports_contract_data(domain)

    def process_seed_training_data(self):
        for domain in domains:
            print("textrank.extract_keyword_seed_data():" + domain)
            self.textrank.extract_keyword_seed_data(domain) 

            print("textrank.load_seed_to_training_data_batch():" + domain)
            self.data_load.load_seed_to_training_data_batch(domain) 

    def process_keyword_model(self):
        for domain in domains:
            print("key_classifier.prepare_training_data():" + domain)
            self.key_classifier.prepare_training_data(domain)

            print("key_classifier.train_model():" + domain)
            self.key_classifier.train_model(domain)

            print("key_classifier.process_contract_data():" + domain)
            self.key_classifier.process_contract_data(domain)

    def process_transformer_model(self):
        for domain in domains:
            print("class_service.prepare_train_dataset():" + domain)
            self.class_service.prepare_train_dataset(domain)

            print("class_service.training():" + domain)
            self.class_service.training(domain)    

            print("class_service.process_contract_training_data_eval():" + domain)
            self.class_service.process_contract_training_data_eval(domain)

            print("risk_class.process_keyword_polarity():" , domain)
            self.risk_class.process_keyword_polarity(domain)

    def evaluate_results(self):
        for domain in domains:
            print ("key_classifier.Keyword Classifier Accuracy: " + domain)
            self.key_classifier.evaluate_model(domain) 
            
            print ("class_service.Transformer Classifier Accuracy: " + domain)
            self.class_service.evalute_model(domain)

if __name__ == '__main__': 
    dbloader = Data_ETL_Pipeline()
    #dbloader.create_dataset()
    #dbloader.load_file_data()
    #dbloader.process_seed_training_data() 
    dbloader.process_keyword_model()
    dbloader.process_transformer_model()
    dbloader.evaluate_results()


/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


key_classifier.prepare_training_data():esg
DB Pool Created.


ERROR:root:Traceback (most recent call last):
  File "/Users/saurabhkaushik/Workspace/lca-ai-services/app/Keyword_Classifier.py", line 52, in train_model
    self.trainDF['text'], self.trainDF['label'])
  File "/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/sklearn/pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/sklearn/pipeline.py", line 355, in _fit
    **fit_params_steps[name],
  File "/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/joblib/memory.py", line 349, in __call__
    return self.func(*args, **kwargs)
  File "/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/sklearn/pipeline.py", line 893, in _fit_transform_one
    res = transformer.fit_transform(X, y, **fit_params)
  File "/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/sklearn/featur

Series([], Name: label, dtype: float64)
key_classifier.train_model():esg
key_classifier.process_contract_data():esg
Query:  -1 record(s) affected
key_classifier.prepare_training_data():liabilities
SELECT * from seed_data where domain='liabilities';
0         current liabilities
1         current liabilities
2         current liabilities
3         current liabilities
4         current liabilities
                ...          
705    contingent liabilities
706    contingent liabilities
707    contingent liabilities
708    contingent liabilities
709    contingent liabilities
Name: label, Length: 710, dtype: object
key_classifier.train_model():liabilities
key_classifier.process_contract_data():liabilities
Filename: IBM_Annual_Report_2002.txt
Sentences :  He nurtured a unique culture, a progressive set of values and the aspiration to make a difference in the world. 
    , Result :  non-current liabilities , P_Score :  100.0
Sentences :  Under Lou GerstnerÕs leadership, the people of IBM reb

In [ ]:
from app.common.GCP_Storage import GCP_Storage

import os 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './config/gcp/genuine-wording-key.json'

domains = ['liabilities', 'esg']
bucket_name = 'lca_dev'
loader = GCP_Storage(domains, bucket_name)

#loader.setup_bucket()
#loader.upload_models()
#loader.download_models()
#loader.download_seed_data()
#loader.upload_seed_data()